In [220]:
import pandas as pd
import numpy as np
df = pd.read_csv('horse_data.csv', header = None)
#переименовываем необходимые столбцы
df = df.rename({
    0: 'surgery',
    1: 'Age',
    3:'rectal_temperature',
    4:'pulse',
    5:'respiratory_rate',
    6:'temperature_of_extremities',
    10:'pain',
    22:'outcome'
}, axis=1)
#Удаляем ненужные
for i in range(28):
    try:
        df = df.drop(i, axis = 1)
    except:
        continue
#Переводим в числовые типы данных
for i in list(df.columns):
    if i != 'rectal_temperature':
        df[i] = df.loc[df[i]!='?'].apply(lambda x: int(x[i]), axis=1)
    else:
        df[i] = df.loc[df[i]!='?'].apply(lambda x: float(x[i]), axis=1)
#заполняем пропуски Nan      
df = df.mask(df == '?')
df.describe()

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
count,299.000000,300.000000,240.000000,276.000000,242.000000,244.000000,245.00000,299.000000
mean,1.397993,1.640000,38.167917,71.913043,30.417355,2.348361,2.95102,1.551839
std,0.490305,2.173972,0.732289,28.630557,17.642231,1.045054,1.30794,0.737187
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.00000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,18.500000,1.000000,2.00000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,24.500000,3.000000,3.00000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.00000,2.000000
max,2.000000,9.000000,40.800000,184.000000,96.000000,4.000000,5.00000,3.000000


# Выбросы

In [221]:
#По столбцу "surgery" все значения корректные (1 или 2)
# По столбцу "Age" есть значения(9), которые не указаны в описании(может принимать только 1 или 2),
#при этом ни одного значения (2), скорее всего была техническая ошибка
#заменим все (9) на (2)
print(df.loc[(df['Age'] !=1)].describe())
df.loc[df['Age'] == 9, 'Age'] = 2
df.describe()

         surgery   Age  rectal_temperature       pulse  respiratory_rate  \
count  24.000000  24.0           20.000000   22.000000         20.000000   
mean    1.250000   9.0           38.645000  123.363636         54.600000   
std     0.442326   0.0            0.490408   28.628234         23.904195   
min     1.000000   9.0           38.000000   84.000000         20.000000   
25%     1.000000   9.0           38.275000  100.000000         34.500000   
50%     1.000000   9.0           38.600000  124.000000         49.000000   
75%     1.250000   9.0           38.925000  144.500000         74.000000   
max     2.000000   9.0           39.700000  184.000000         96.000000   

       temperature_of_extremities       pain    outcome  
count                   16.000000  17.000000  24.000000  
mean                     2.125000   3.117647   1.541667  
std                      1.087811   1.218726   0.588230  
min                      1.000000   1.000000   1.000000  
25%                      

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
count,299.000000,300.000000,240.000000,276.000000,242.000000,244.000000,245.00000,299.000000
mean,1.397993,1.080000,38.167917,71.913043,30.417355,2.348361,2.95102,1.551839
std,0.490305,0.271746,0.732289,28.630557,17.642231,1.045054,1.30794,0.737187
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.00000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,18.500000,1.000000,2.00000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,24.500000,3.000000,3.00000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.00000,2.000000
max,2.000000,2.000000,40.800000,184.000000,96.000000,4.000000,5.00000,3.000000


In [222]:
#Определим выбросы по межквартильному размаху "rectal temperature"
#Первое значение подозрительное, при низкой температуре очень высокий пульс
q1 = df['rectal_temperature'].quantile(0.25)
q3 = df['rectal_temperature'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df[df['rectal_temperature'].between(lower_bound, upper_bound, inclusive=True)].sort_values('rectal_temperature')
pd.concat([df, remove_outliers]).drop_duplicates(keep=False).dropna(subset=['rectal_temperature'])

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
20,1.0,1,39.9,72.0,60.0,1.0,5.0,1.0
44,1.0,1,35.4,140.0,24.0,3.0,4.0,3.0
54,2.0,1,40.3,114.0,36.0,3.0,2.0,3.0
75,1.0,2,39.7,100.0,NaN,3.0,2.0,3.0
80,1.0,1,36.4,98.0,35.0,3.0,4.0,2.0
91,2.0,1,40.3,114.0,36.0,3.0,2.0,2.0
99,2.0,1,39.6,108.0,51.0,3.0,2.0,1.0
118,1.0,1,36.5,78.0,30.0,1.0,5.0,1.0
141,2.0,1,36.0,42.0,30.0,NaN,NaN,2.0
238,2.0,1,36.1,88.0,NaN,3.0,3.0,3.0


In [223]:
#Определим выбросы по межквартильному размаху "pulse"
q1 = df['pulse'].quantile(0.25)
q3 = df['pulse'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df[df['pulse'].between(lower_bound, upper_bound, inclusive=True)].sort_values('pulse')
pd.concat([df, remove_outliers]).drop_duplicates(keep=False).dropna(subset=['pulse'])

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
3,1.0,2,39.1,164.0,84.0,4.0,2.0,2.0
41,2.0,2,39.0,150.0,72.0,NaN,NaN,1.0
55,1.0,2,38.6,160.0,20.0,3.0,3.0,2.0
255,1.0,2,38.8,184.0,84.0,1.0,4.0,2.0
275,1.0,2,38.8,150.0,50.0,1.0,5.0,2.0


In [224]:
#Тот же выброс, удалим эту строку
df = df.drop(df[df.rectal_temperature == 35.4].index)

In [225]:
#Определим выбросы по межквартильному размаху "respiratory_rate"
q1 = df['respiratory_rate'].quantile(0.25)
q3 = df['respiratory_rate'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df[df['respiratory_rate'].between(lower_bound, upper_bound, inclusive=True)].sort_values('respiratory_rate')
pd.concat([df, remove_outliers]).drop_duplicates(keep=False).dropna(subset=['respiratory_rate']).sort_values('respiratory_rate')

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
120,1.0,1,39.4,54.0,66.0,1.0,2.0,1.0
103,1.0,2,38.0,140.0,68.0,1.0,3.0,1.0
125,1.0,1,38.0,42.0,68.0,4.0,3.0,1.0
265,2.0,1,NaN,56.0,68.0,3.0,3.0,3.0
229,1.0,2,38.5,120.0,70.0,NaN,NaN,1.0
295,1.0,1,NaN,120.0,70.0,4.0,2.0,3.0
41,2.0,2,39.0,150.0,72.0,NaN,NaN,1.0
82,1.0,2,38.1,100.0,80.0,3.0,3.0,1.0
84,1.0,1,37.8,60.0,80.0,1.0,2.0,1.0
208,1.0,1,37.8,88.0,80.0,3.0,NaN,3.0


In [226]:
#Удалим эти строки
df = df.drop(df[df.respiratory_rate >= 58.0	].index)

In [227]:
#В оставшихся столбцах все значения корректны
df.describe()

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
count,276.000000,277.000000,219.000000,253.000000,219.000000,226.000000,230.000000,276.000000
mean,1.423913,1.054152,38.137900,69.019763,25.977169,2.340708,2.943478,1.550725
std,0.495075,0.226726,0.705533,25.482246,10.468144,1.030136,1.322075,0.729358
min,1.000000,1.000000,36.000000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,37.750000,48.000000,18.000000,1.000000,2.000000,1.000000
50%,1.000000,1.000000,38.100000,60.000000,24.000000,3.000000,3.000000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,32.000000,3.000000,4.000000,2.000000
max,2.000000,2.000000,40.800000,160.000000,52.000000,4.000000,5.000000,3.000000


# Заполнение пропусков

In [228]:
#удалим строки с 3 и более пропусками
df = df.dropna(thresh=6)

In [229]:
#столбец "surgery"
df[df['surgery'].isnull()]

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
132,NaN,1,38.0,48.0,20.0,3.0,4.0,NaN


In [230]:
#заполним медианным значением(оно же модальное, так как всего два значения)
df['surgery'].fillna(df['surgery'].median(), inplace=True)

In [232]:
#Не группируем по "Age", так как мало строк для параметра (2)
#столбец "rectal_temperature"
#По группам нет особово различия, поэтому заполним медианным
print(df.groupby('surgery')['rectal_temperature'].median())
print(df.groupby('temperature_of_extremities')['rectal_temperature'].median())
print(df.groupby('pain')['rectal_temperature'].median())
print(df.groupby('outcome')['rectal_temperature'].median())
df['rectal_temperature'].fillna(df['rectal_temperature'].median(), inplace=True)
df.describe()

surgery
1.0    38.1
2.0    38.2
Name: rectal_temperature, dtype: float64
temperature_of_extremities
1.0    38.1
2.0    38.2
3.0    38.1
4.0    38.3
Name: rectal_temperature, dtype: float64
pain
1.0    38.20
2.0    38.30
3.0    38.20
4.0    38.05
5.0    38.00
Name: rectal_temperature, dtype: float64
outcome
1.0    38.2
2.0    38.0
3.0    38.1
Name: rectal_temperature, dtype: float64


,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
count,255.000000,255.00000,255.000000,246.000000,217.000000,218.000000,223.000000,254.000000
mean,1.431373,1.05098,38.132941,69.093496,25.976959,2.330275,2.919283,1.547244
std,0.496242,0.22039,0.652986,25.627381,10.472381,1.038920,1.309265,0.735688
min,1.000000,1.00000,36.000000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.000000,1.00000,37.800000,48.000000,18.000000,1.000000,2.000000,1.000000
50%,1.000000,1.00000,38.100000,60.000000,24.000000,3.000000,3.000000,1.000000
75%,2.000000,1.00000,38.500000,88.000000,32.000000,3.000000,4.000000,2.000000
max,2.000000,2.00000,40.800000,160.000000,52.000000,4.000000,5.000000,3.000000


In [234]:
#столбец "pulse"
#заполним медианным по группе "temperature_of_extremities",  что было и логично предположить, 
#что с чем ниже пульс, тем хуже кровообращение
print(df.groupby('surgery')['pulse'].median())
print(df.groupby('temperature_of_extremities')['pulse'].median())
print(df.groupby('pain')['pulse'].median())
print(df.groupby('outcome')['pulse'].median())
df['pulse'].fillna(df.groupby('temperature_of_extremities')['pulse'].transform('median'), inplace=True)
df.describe()

surgery
1.0    68.0
2.0    54.0
Name: pulse, dtype: float64
temperature_of_extremities
1.0    52.0
2.0    54.0
3.0    80.0
4.0    88.0
Name: pulse, dtype: float64
pain
1.0    48.0
2.0    72.0
3.0    60.0
4.0    77.5
5.0    90.0
Name: pulse, dtype: float64
outcome
1.0    52.0
2.0    86.0
3.0    81.0
Name: pulse, dtype: float64


,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
count,255.000000,255.00000,255.000000,255.000000,217.000000,218.000000,223.000000,254.000000
mean,1.431373,1.05098,38.132941,69.431373,25.976959,2.330275,2.919283,1.547244
std,0.496242,0.22039,0.652986,25.301221,10.472381,1.038920,1.309265,0.735688
min,1.000000,1.00000,36.000000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.000000,1.00000,37.800000,48.000000,18.000000,1.000000,2.000000,1.000000
50%,1.000000,1.00000,38.100000,64.000000,24.000000,3.000000,3.000000,1.000000
75%,2.000000,1.00000,38.500000,88.000000,32.000000,3.000000,4.000000,2.000000
max,2.000000,2.00000,40.800000,160.000000,52.000000,4.000000,5.000000,3.000000


In [236]:
#столбец "respiratory_rate"
# Заполним медианным по группе "pain", что чем больнее лошади, тем чаще дыхание
print(df.groupby('surgery')['respiratory_rate'].median())
print(df.groupby('temperature_of_extremities')['respiratory_rate'].median())
print(df.groupby('pain')['respiratory_rate'].median())
print(df.groupby('outcome')['respiratory_rate'].median())
df['respiratory_rate'].fillna(df.groupby('pain')['respiratory_rate'].transform('median'), inplace=True)
df.describe()

surgery
1.0    24.0
2.0    24.0
Name: respiratory_rate, dtype: float64
temperature_of_extremities
1.0    24.0
2.0    30.0
3.0    28.0
4.0    30.0
Name: respiratory_rate, dtype: float64
pain
1.0    20.0
2.0    24.0
3.0    24.0
4.0    30.0
5.0    28.0
Name: respiratory_rate, dtype: float64
outcome
1.0    24.0
2.0    28.0
3.0    24.0
Name: respiratory_rate, dtype: float64


,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
count,255.000000,255.00000,255.000000,255.000000,254.000000,218.000000,223.000000,254.000000
mean,1.431373,1.05098,38.132941,69.431373,25.893701,2.330275,2.919283,1.547244
std,0.496242,0.22039,0.652986,25.301221,9.759586,1.038920,1.309265,0.735688
min,1.000000,1.00000,36.000000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.000000,1.00000,37.800000,48.000000,20.000000,1.000000,2.000000,1.000000
50%,1.000000,1.00000,38.100000,64.000000,24.000000,3.000000,3.000000,1.000000
75%,2.000000,1.00000,38.500000,88.000000,31.500000,3.000000,4.000000,2.000000
max,2.000000,2.00000,40.800000,160.000000,52.000000,4.000000,5.000000,3.000000


In [238]:
#столбец "temperature_of_extremities"
# Заполним медианным по группе "surgery",распределим среднние(2 и 3) по признаку хирургического вмешательства
print(df.groupby('surgery')['temperature_of_extremities'].median())
print(df.groupby('pain')['temperature_of_extremities'].median())
print(df.groupby('outcome')['temperature_of_extremities'].median())
df['temperature_of_extremities'].fillna(df.groupby('surgery')['temperature_of_extremities'].transform('median'), inplace=True)
df.describe()

surgery
1.0    3.0
2.0    2.0
Name: temperature_of_extremities, dtype: float64
pain
1.0    2.0
2.0    3.0
3.0    2.0
4.0    3.0
5.0    3.0
Name: temperature_of_extremities, dtype: float64
outcome
1.0    2.0
2.0    3.0
3.0    3.0
Name: temperature_of_extremities, dtype: float64


,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
count,255.000000,255.00000,255.000000,255.000000,254.000000,255.000000,223.000000,254.000000
mean,1.431373,1.05098,38.132941,69.431373,25.893701,2.380392,2.919283,1.547244
std,0.496242,0.22039,0.652986,25.301221,9.759586,0.984330,1.309265,0.735688
min,1.000000,1.00000,36.000000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.000000,1.00000,37.800000,48.000000,20.000000,1.000000,2.000000,1.000000
50%,1.000000,1.00000,38.100000,64.000000,24.000000,3.000000,3.000000,1.000000
75%,2.000000,1.00000,38.500000,88.000000,31.500000,3.000000,4.000000,2.000000
max,2.000000,2.00000,40.800000,160.000000,52.000000,4.000000,5.000000,3.000000


In [240]:
#столбец "pain"
# Заполним также медианными по "surgery", с хирургическим вмешательством больнее
print(df.groupby('surgery')['pain'].median())
print(df.groupby('temperature_of_extremities')['pain'].median())
print(df.groupby('outcome')['pain'].median())
df['pain'].fillna(df.groupby('surgery')['pain'].transform('median'), inplace=True)
df.describe()

surgery
1.0    3.0
2.0    2.0
Name: pain, dtype: float64
temperature_of_extremities
1.0    3.0
2.0    2.0
3.0    3.0
4.0    3.0
Name: pain, dtype: float64
outcome
1.0    3.0
2.0    4.0
3.0    3.0
Name: pain, dtype: float64


,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
count,255.000000,255.00000,255.000000,255.000000,254.000000,255.000000,255.000000,254.000000
mean,1.431373,1.05098,38.132941,69.431373,25.893701,2.380392,2.874510,1.547244
std,0.496242,0.22039,0.652986,25.301221,9.759586,0.984330,1.242274,0.735688
min,1.000000,1.00000,36.000000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.000000,1.00000,37.800000,48.000000,20.000000,1.000000,2.000000,1.000000
50%,1.000000,1.00000,38.100000,64.000000,24.000000,3.000000,3.000000,1.000000
75%,2.000000,1.00000,38.500000,88.000000,31.500000,3.000000,4.000000,2.000000
max,2.000000,2.00000,40.800000,160.000000,52.000000,4.000000,5.000000,3.000000


In [242]:
#Осталась две строчки, их можно удалить
df = df.dropna()
df.describe()

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
count,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000
mean,1.434783,1.051383,38.132016,69.553360,25.916996,2.383399,2.869565,1.549407
std,0.496711,0.221216,0.655110,25.358269,9.771853,0.983616,1.245142,0.736337
min,1.000000,1.000000,36.000000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,20.000000,1.000000,2.000000,1.000000
50%,1.000000,1.000000,38.100000,64.000000,24.000000,3.000000,3.000000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,32.000000,3.000000,4.000000,2.000000
max,2.000000,2.000000,40.800000,160.000000,52.000000,4.000000,5.000000,3.000000
